In [1]:
import numpy as np
import os
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

datFile = training + "shape_predictor_68_face_landmarks.dat"

In [3]:
batch_size = 32 
img_height = 224
img_width = 224 

In [4]:
training_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)
validation_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred', 
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 399839 files belonging to 7 classes.
Found 50947 files belonging to 7 classes.


In [5]:
base_model = keras.applications.EfficientNetB4(input_shape=(224,224,3), include_top = False, weights="imagenet")

In [6]:
base_model.trainable=False

In [7]:
inputs = keras.Input(shape=(224,224,3))
x = base_model(inputs, training=False)

In [8]:
avg = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(7, activation='softmax')(avg)
model = keras.Model(inputs=inputs, outputs=outputs)

In [9]:
model.compile(optimizer="Adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(training_s, validation_data=validation_s, epochs=25)

Epoch 1/25
12495/12495 [==============================] - 1981s 157ms/step - loss: 1.3707 - accuracy: 0.4747 - val_loss: 1.3041 - val_accuracy: 0.5082
Epoch 2/25
12495/12495 [==============================] - 1949s 156ms/step - loss: 1.2814 - accuracy: 0.5137 - val_loss: 1.2604 - val_accuracy: 0.5226
Epoch 3/25
12495/12495 [==============================] - 1972s 158ms/step - loss: 1.2562 - accuracy: 0.5240 - val_loss: 1.2492 - val_accuracy: 0.5302
Epoch 4/25
12495/12495 [==============================] - 1978s 158ms/step - loss: 1.2431 - accuracy: 0.5296 - val_loss: 1.2404 - val_accuracy: 0.5347
Epoch 5/25
12495/12495 [==============================] - 1972s 158ms/step - loss: 1.2341 - accuracy: 0.5336 - val_loss: 1.2341 - val_accuracy: 0.5368
Epoch 6/25
12495/12495 [==============================] - 1962s 157ms/step - loss: 1.2285 - accuracy: 0.5361 - val_loss: 1.2372 - val_accuracy: 0.5335
Epoch 7/25
12495/12495 [==============================] - 1964s 157ms/step - loss: 1.2240 - ac

In [11]:
model.save('1005_frozen_effiB4.h5')

In [15]:
base_model.trainable = True
for layer in model.layers[1].layers:
    if isinstance(layer, keras.layers.BatchNormalization):
        layer.trainable = False

In [16]:
opt = keras.optimizers.Adam(learning_rate=1e-4)

In [17]:
es = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

In [18]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(training_s, validation_data=validation_s, epochs=10, callbacks=[es])

Epoch 1/10
12495/12495 [==============================] - 9638s 770ms/step - loss: 0.7277 - accuracy: 0.7301 - val_loss: 0.5995 - val_accuracy: 0.7836
Epoch 2/10
12495/12495 [==============================] - 9582s 767ms/step - loss: 0.4706 - accuracy: 0.8285 - val_loss: 0.5687 - val_accuracy: 0.8079
Epoch 3/10
12495/12495 [==============================] - 9544s 764ms/step - loss: 0.3146 - accuracy: 0.8850 - val_loss: 0.6388 - val_accuracy: 0.8098
Epoch 4/10
12495/12495 [==============================] - 9535s 763ms/step - loss: 0.2217 - accuracy: 0.9183 - val_loss: 0.6840 - val_accuracy: 0.8137
Epoch 5/10
12495/12495 [==============================] - 9533s 763ms/step - loss: 0.1707 - accuracy: 0.9375 - val_loss: 0.8069 - val_accuracy: 0.8086


In [19]:
model.evaluate(training_s)

12495/12495 [==============================] - 1741s 139ms/step - loss: 0.3697 - accuracy: 0.8656


[0.36973732709884644, 0.865623414516449]

In [20]:
model.evaluate(validation_s)

1593/1593 [==============================] - 224s 141ms/step - loss: 0.5687 - accuracy: 0.8079


[0.5687124133110046, 0.8078787922859192]

In [21]:
model.save("1005_B4_eval.h5")